<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#读入数据" data-toc-modified-id="读入数据-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>读入数据</a></span></li><li><span><a href="#根据用户id和物品id构建rating矩阵" data-toc-modified-id="根据用户id和物品id构建rating矩阵-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>根据用户id和物品id构建rating矩阵</a></span><ul class="toc-item"><li><span><a href="#将数据中的空值填充为0" data-toc-modified-id="将数据中的空值填充为0-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>将数据中的空值填充为0</a></span></li></ul></li><li><span><a href="#基于用户的协同过滤" data-toc-modified-id="基于用户的协同过滤-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>基于用户的协同过滤</a></span></li><li><span><a href="#基于物品的推荐" data-toc-modified-id="基于物品的推荐-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>基于物品的推荐</a></span></li></ul></div>

# 读入数据

In [3]:
#导包
import pandas as pd
import numpy as np

In [4]:
# 读入数据
df = pd.read_csv("ml-100k/u.data",delimiter='\t',header=None)
# 修改列名
df.columns=['用户id','物品id','喜好程度','时间']

In [5]:
df.shape

(100000, 4)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   用户id    100000 non-null  int64
 1   物品id    100000 non-null  int64
 2   喜好程度    100000 non-null  int64
 3   时间      100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [7]:
df.head()

,用户id,物品id,喜好程度,时间
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
del df['时间']
df.head()

,用户id,物品id,喜好程度
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


# 根据用户id和物品id构建rating矩阵

In [9]:
# 使用 pivot函数
df_pivot = df.pivot(index="用户id",columns="物品id",values="喜好程度")
df_pivot

物品id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
用户id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_pivot.shape #用户-电影 评分矩阵

(943, 1682)

## 将数据中的空值填充为0

In [15]:
freq = df_pivot.fillna(0)
freq_matrix = freq.values
freq_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

# 基于用户的协同过滤

In [12]:
#导入相关库
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 构建推荐指数函数，第item_id个物品对第user_id个用户的推荐分值，看最相似的k个用户
def Recommendation_mean(user_id,item_id,similar,k=10):
    """减去平均数的计算方法"""
    score = 0
    weight = 0
    user_id_action = freq_matrix[user_id,:]      #用户user_id 对所有商品的行为评分  
    item_id_action = freq_matrix[:,item_id]      #物品item_id 得到的所有用户评分  
    user_id_similar = similar[user_id,:]      #用户user_id 对所有用户的相似度    
    similar_index = np.argsort(user_id_similar)[-(k+1):-1]  #最相似的k个用户的index（除了自己）
   
    user_id_i_mean = np.sum(user_id_action)/user_id_action[user_id_action!=0].size#
    for j in similar_index : #0,3
        if item_id_action[j]!=0: 
            user_id_j_action = freq_matrix[j,:]
            user_id_j_mean = np.sum(user_id_j_action)/user_id_j_action[user_id_j_action!=0].size
            score += user_id_similar[j]*(item_id_action[j]-user_id_j_mean)
#             print(score)
            weight += abs(user_id_similar[j])
#             print(weight)
    if weight==0:  
        return 0
    else:
        return user_id_i_mean + score/float(weight)
    
#构建预测函数
def predict_mean(user_similar):
    """预测函数的功能: 传入相似度矩阵, 通过对每个用户和每个物品进行计算, 计算出一个推荐矩阵"""
    user_count = freq_matrix.shape[0]#用户数
    item_count = freq_matrix.shape[1]#商品数
    predic_matrix = np.zeros((user_count,item_count))
    for user_id in range(user_count):
        for item_id in range(item_count):
            if freq_matrix[user_id,item_id] == 0:
                #print (user_id,item_id)
                predic_matrix[user_id,item_id] = Recommendation_mean(user_id,item_id,user_similar)
    return predic_matrix  


def get_topk(group,n):
    return group.sort_values("推荐指数",ascending=False)[:n]


#计算相似性矩阵
user_similar=cosine_similarity(freq_matrix)

#得到预测分数矩阵
user_prediction_matrix = predict_mean(user_similar)

#得到预测分数的dataframe
recommendation_df = pd.DataFrame(user_prediction_matrix,columns=freq.columns,index=freq.index)

recommendation_df_2 = recommendation_df.stack().reset_index()
recommendation_df_2.rename(columns={0:"推荐指数"},inplace=True)

recommendation_df_grouped = recommendation_df_2.groupby("用户id")

#获得top n推荐
topk = recommendation_df_grouped.apply(get_topk,n=3) #推荐几个

topk = topk.drop(["用户id"],axis=1)
topk.index = topk.index.droplevel(1)
topk.reset_index(inplace=True)
topk

,用户id,物品id,推荐指数
0,1,855,5.365449
1,1,963,5.365449
2,1,1007,5.244592
3,2,754,5.834677
4,2,1405,5.487455
...,...,...,...
2824,942,727,5.845031
2825,942,1142,5.845031
2826,943,1019,5.116894
2827,943,272,4.952071


# 基于物品的推荐

In [18]:
#导入相关库
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


#构建一个基于物品的推荐
def Recommendation_s(uid,iid,similar,k=10): #2,2，第2个物品对第2个用户的推荐分值
    score = 0
    weight = 0
    uid_action = freq_matrix[uid,:]      #用户uid 对所有商品的行为评分  
    iid_action = freq_matrix[:,iid]      #物品iid 得到的所有用户评分  
    
    iid_sim = similar[iid,:]      #商品iid 和所有商品的相似度    
    sim_indexs = np.argsort(iid_sim)[-(k+1):-1]  #最相似的k个物品的index（除了自己）
    
    iid_i_mean = np.sum(iid_action)/iid_action[iid_action!=0].size
    
    for j in sim_indexs : #0,6
        if uid_action[j]!=0:  #表示用户用过该物品，用户对该物品有过评分
            iid_j_action = freq_matrix[:,j]
            iid_j_mean = np.sum(iid_j_action)/iid_j_action[iid_j_action!=0].size
            score += iid_sim[j]*(uid_action[j]-iid_j_mean)
            weight += abs(iid_sim[j])

    if weight==0:  
        return 0
    else:
        return iid_i_mean + score/float(weight)

    
#构建预测函数
def predict_mean(similar):
    user_cnt = freq_matrix.shape[0]
    item_cnt = freq_matrix.shape[1]
    pred = np.zeros((user_cnt,item_cnt))
    for uid in range(user_cnt):
        for iid in range(item_cnt):
            if freq_matrix[uid,iid] == 0: #表示用户没有用过该物品
                #print (uid,iid)
                pred[uid,iid] = Recommendation_s(uid,iid,similar)
    return pred

def get_topk(group,n):
    return group.sort_values("推荐指数",ascending=False)[:n]


#计算相似性矩阵
item_similar=cosine_similarity(freq_matrix.T)#计算物品的相似性

#得到预测分数矩阵
user_prediction_matrix = predict_mean(item_similar)

#得到预测分数的dataframe
recommendation_df = pd.DataFrame(user_prediction_matrix,columns=freq.columns,index=freq.index)

recommendation_df_2 = recommendation_df.stack().reset_index()
recommendation_df_2.rename(columns={0:"推荐指数"},inplace=True)

recommendation_df_grouped = recommendation_df_2.groupby("用户id")

#获得top n推荐
topk = recommendation_df_grouped.apply(get_topk,n=3) #推荐几个

topk = topk.drop(["用户id"],axis=1)
topk.index = topk.index.droplevel(1)
topk.reset_index(inplace=True)
topk

,用户id,物品id,推荐指数
0,1,711,5.794872
1,1,1536,5.434116
2,1,1525,5.266667
3,2,12,5.230256
4,2,694,5.191776
...,...,...,...
2824,942,114,5.243402
2825,942,12,5.236278
2826,943,1467,6.555556
2827,943,853,5.378129
